In [16]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

In [17]:
names_df = pd.read_csv('./Seoul.csv', index_col=0)
names_df

,0
0,강남구 논현동 숯칼 도산 숯닭갈비
1,강남구 논현동 카페 베이직먼트(cafe basicment)
2,강남구 논현동 곱분이
3,강남구 논현동 깐부치킨
4,강남구 논현동 백순대본가새맛
...,...
2011,강남구 논현동 인터렉트
2012,강남구 논현동 류몽민
2013,강남구 논현동 류창희국수人
2014,강남구 논현동 357텐


In [18]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver'

source_url = "https://map.kakao.com/"

page_urls = []

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for name in names_df['0']:
    try:
        driver.get(source_url)

        search_bar = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
        search_bar.send_keys(name)

        searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
        driver.execute_script("arguments[0].click();", searchbutton)

        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="info.search.place.list"]/li[2]/div[3]/strong/a[1]')))

        html = driver.page_source

        soup = BeautifulSoup(html, "html.parser")
        moreview = soup.find(name="a", attrs={"class":"moreview"})
    
        page_url = moreview.get("href")
        print(page_url)
        page_urls.append(page_url)
    except:
        pass

driver.close()

https://place.map.kakao.com/18260687
https://place.map.kakao.com/20944140
https://place.map.kakao.com/2023215976
https://place.map.kakao.com/1630603393
https://place.map.kakao.com/8188651
https://place.map.kakao.com/18865294
https://place.map.kakao.com/13314189
https://place.map.kakao.com/12097795
https://place.map.kakao.com/1212472221
https://place.map.kakao.com/11572197
https://place.map.kakao.com/1390671305
https://place.map.kakao.com/1886246975
https://place.map.kakao.com/2124242484
https://place.map.kakao.com/1462857171
https://place.map.kakao.com/27548800
https://place.map.kakao.com/313365189
https://place.map.kakao.com/679866906
https://place.map.kakao.com/1276882929
https://place.map.kakao.com/332630576
https://place.map.kakao.com/1348944575
https://place.map.kakao.com/1935554229
https://place.map.kakao.com/2000969854
https://place.map.kakao.com/12698826
https://place.map.kakao.com/123889749
https://place.map.kakao.com/1178003813
https://place.map.kakao.com/1311151895
https://p

In [19]:
columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for page_url in page_urls:
    try:
        driver.get(page_url)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')))
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names = soup.find(name='h2', attrs={"class":"tit_location"})
        
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
    
    
        for rate, review in zip(rates, reviews):
            if review != ' ':
                row = [names.text, rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=review_df.columns)
                review_df = review_df.append(series, ignore_index=True)

        for button_num in range(2, 6):
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            driver.implicitly_wait(5)
            
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                if review != ' ':
                    row = [names.text, rate.text[0], review.find(name="span").text]
                    series = pd.Series(row, index=review_df.columns)
                    review_df = review_df.append(series, ignore_index=True)
    except:
        pass

    
driver.close()

In [20]:
review_df['review'].replace(' ', np.nan, inplace=True)
review_data = review_df.dropna(subset=['review'])
review_data

,name,score,review
0,깐부치킨 강남구청역3번출구점,2,맛도 평균이하이고 좀 불친절함 장소나 넓은것 야외로 오픈되어있는건 좋은데
1,광주식당,4,신선 깔끔함 카드 됨
2,광주식당,1,"현금 없으면 가지 마세요, 현금만 받는다고 공지를 하던까? 맛있게 잘 먹고 당황함.^^♡"
3,컨트롤브이,5,사장님도 정말 친절하시고 음료도 맛있어요 카페도 엄청 깔끔하구요.. 무엇보다 하루가...
4,고향집,5,여기 황태탕은… 정말 최고임
...,...,...,...
2914,인터랙트,1,뜨아는 바로 가져갔는데도 차갑고 바닐라라떼도 무슨맛인지 뜨뜨미지근 값은 너무 비쌈
2915,인터랙트,1,무설탕 글루텐프리여도 맛이없으면 무슨소용… 케이크가 너무 맛이없네요…
2916,인터랙트,1,
2917,인터랙트,1,케이크 너무 맛없어서 놀랬어요…


In [21]:
review_data.to_csv("review_data.csv", index=False)